In [ ]:
#!/usr/bin/env python3
"""Visualization script for Go1 with height scanner."""
import os
# Tell XLA to use Triton GEMM
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags
os.environ['MUJOCO_GL'] = 'egl'

import jax
import jax.numpy as jp
import numpy as np
import mujoco
import cv2
from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as IPyimage, display
from custom_env import Joystick, default_config
from io import BytesIO

In [ ]:

# Set up visualization options
scene_option = mujoco.MjvOption()
scene_option.geomgroup[2] = True   # Show visual geoms
scene_option.geomgroup[3] = False  # Hide collision geoms
scene_option.geomgroup[5] = True   # Show sites (including height scanner visualization)
scene_option.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True  # Show contact points
scene_option.flags[mujoco.mjtVisFlag.mjVIS_RANGEFINDER] = True
print("Creating Visualization...")

# We will no longer use the registry, but directly load our custom XML and model.
# We will load the "debug" version here, which adds an additional wall *under* the robot
# We add this to understand that changing walls correctly affects collision + raycasting.
xml_path = 'custom_env.xml' # 'custom_env_debug_wall.xml'
env = Joystick(xml_path=xml_path, config=default_config())

# NOTE: For this test, we manually set init_q z position high to avoid collisions with walls 
# env._init_q = env._init_q.at[2].set(1.0)

# JIT compile the functions for speed
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)
jit_terrain_height = jax.jit(env._get_torso_terrain_height)

# Initialize
key = jax.random.PRNGKey(15)

print("Running simulation...")

# Perform multiple random resets to show different wall configurations
num_resets = 1
steps_per_reset = 50  # More steps to show movement

for reset_idx in range(num_resets):
    print(f"Creating GIF {reset_idx+1}/{num_resets}")

    # Randomize wall heights for this reset
    # set_wall_heights(env, range_min=0.05, range_max=0.4)

    # Generate new random key for this reset
    key, reset_key = jax.random.split(key)

    # Reset to new random position
    state = jit_reset(reset_key)

    # Collect frames and terrain heights for this reset
    rollout = []
    terrain_heights = []

    for step in range(steps_per_reset):
        rollout.append(state)

        # Calculate terrain height for this state
        terrain_height = jit_terrain_height(state.data)
        terrain_heights.append(float(terrain_height))

        # Check for NaN/inf in reward
        if not jp.isfinite(state.reward):
            print(f"    WARNING: Non-finite reward detected!")
            print(f"    Reward value: {state.reward}")
            raise ValueError("Non-finite reward encountered during simulation")

        # Use small random actions to show some movement
        action_key, key = jax.random.split(key)
        action = jax.random.normal(action_key, (env.action_size,)) * 0.1
        state = jit_step(state, action)

    # Debug prints: check wall positions from simulation data
    current_data = rollout[-1].data  # Get the latest state
    print(f"\nReset {reset_idx+1} - Wall positions:")
    for i, wall_geom_id in enumerate(env._wall_geom_ids):
        wall_pos = current_data.geom_xpos[wall_geom_id]
        print(f"  Wall {i}: geom_xpos[z]={wall_pos[2]:.3f}")
    print()

    # Render frames for this reset
    print(f"    Rendering {len(rollout)} frames...")

    frames = env.render(
        rollout,
        camera="track",  # Use tracking camera
        scene_option=scene_option,
        width=480,
        height=360,
    )

    # # Create GIF for this reset
    # gif_filename = f'wall_randomization_{reset_idx+1:02d}.gif'
    # print(f"    Saving GIF to {gif_filename}...")

    # # Convert frames to PIL Images and add terrain height overlay
    # pil_images = []
    # for i, frame in enumerate(frames):
    #     pil_image = Image.fromarray(frame)

    #     # Add terrain height overlay
    #     draw = ImageDraw.Draw(pil_image)
    #     terrain_height_text = f"Terrain Height: {terrain_heights[i]:.3f}m"

    #     # Try to use a default font, fallback to default if not available
    #     try:
    #         font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 20)
    #     except:
    #         font = ImageFont.load_default()

    #     # Draw background rectangle for better text visibility
    #     text_bbox = draw.textbbox((0, 0), terrain_height_text, font=font)
    #     text_width = text_bbox[2] - text_bbox[0]
    #     text_height = text_bbox[3] - text_bbox[1]

    #     # Position text in top-left corner with padding
    #     x, y = 10, 10
    #     draw.rectangle([x-5, y-5, x+text_width+5, y+text_height+5], fill=(0, 0, 0, 128))
    #     draw.text((x, y), terrain_height_text, fill=(255, 255, 255), font=font)

    #     pil_images.append(pil_image)
    pil_images = []
    for i, frame in enumerate(frames):
        pil_image = Image.fromarray(frame)

        # Add terrain height overlay
        draw = ImageDraw.Draw(pil_image)
        terrain_height_text = f"Terrain Height: {terrain_heights[i]:.3f}m"

        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 20)
        except:
            font = ImageFont.load_default()

        text_bbox = draw.textbbox((0, 0), terrain_height_text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

        x, y = 10, 10
        draw.rectangle([x-5, y-5, x+text_width+5, y+text_height+5], fill=(0, 0, 0, 128))
        draw.text((x, y), terrain_height_text, fill=(255, 255, 255), font=font)

        pil_images.append(pil_image)

    # Display GIF inline without saving
    buf = BytesIO()
    pil_images[0].save(
        buf,
        format="GIF",
        save_all=True,
        append_images=pil_images[1:],
        duration=80,   # ms per frame
        loop=0
    )
    buf.seek(0)
    display(IPyimage(data=buf.getvalue()))
    

print(f"✓ All {num_resets} GIFs created successfully!")


In [ ]:
# Imports
from typing import Any, Tuple
from brax.training import types
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.types import Params
import flax
import jax
import jax.numpy as jnp
from brax.training.agents.ppo.losses import compute_gae

# Helper Struct for policy and value params
@flax.struct.dataclass
class PPONetworkParams:
  """Contains training state for the learner."""

  policy: Params
  value: Params

# PPO loss function to be implemented
def compute_custom_ppo_loss(
    params: PPONetworkParams,
    normalizer_params: Any,
    data: types.Transition,
    rng: jnp.ndarray,
    ppo_network: ppo_networks.PPONetworks,
    entropy_cost: float = 1e-4,
    discounting: float = 0.9,
    reward_scaling: float = 1.0,
    gae_lambda: float = 0.95,
    clipping_epsilon: float = 0.3,
    normalize_advantage: bool = True,
) -> Tuple[jnp.ndarray, types.Metrics]:
  """Computes PPO loss.

  Args:
    params: Network parameters,
    normalizer_params: Parameters of the normalizer.
    data: Transition that with leading dimension [B, T]. extra fields required
      are ['state_extras']['truncation'] ['policy_extras']['raw_action']
      ['policy_extras']['log_prob']
    rng: Random key
    ppo_network: PPO networks.
    entropy_cost: entropy cost.
    discounting: discounting,
    reward_scaling: reward multiplier.
    gae_lambda: General advantage estimation lambda.
    clipping_epsilon: Policy loss clipping epsilon
    normalize_advantage: whether to normalize advantage estimate

  Returns:
    A tuple (loss, metrics)
  """
  parametric_action_distribution = ppo_network.parametric_action_distribution
  policy_apply = ppo_network.policy_network.apply
  value_apply = ppo_network.value_network.apply

  # Put the time dimension first.
  data = jax.tree_util.tree_map(lambda x: jnp.swapaxes(x, 0, 1), data)
  policy_logits = policy_apply(
      normalizer_params, params.policy, data.observation
  )

  baseline = value_apply(normalizer_params, params.value, data.observation)
  terminal_obs = jax.tree_util.tree_map(lambda x: x[-1], data.next_observation)
  bootstrap_value = value_apply(normalizer_params, params.value, terminal_obs)

  rewards = data.reward * reward_scaling
  truncation = data.extras['state_extras']['truncation']
  termination = (1 - data.discount) * (1 - truncation)

  target_action_log_probs = parametric_action_distribution.log_prob(
      policy_logits, data.extras['policy_extras']['raw_action']
  )
  behaviour_action_log_probs = data.extras['policy_extras']['log_prob']

  vs, advantages = compute_gae(
      truncation=truncation,
      termination=termination,
      rewards=rewards,
      values=baseline,
      bootstrap_value=bootstrap_value,
      lambda_=gae_lambda,
      discount=discounting,
  )
  if normalize_advantage:
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
  
  # TODO: Compute probability ratio between new and old policy
  # Hint: Use jnp.exp() and compute the difference between log probabilities
  rho_s = jnp.exp(target_action_log_probs - behaviour_action_log_probs)
  
  # TODO: Implement the PPO clipped surrogate objective
  # Hint: Compare unclipped (rho_s * advantages) vs clipped version
  # Use jnp.clip() with clipping_epsilon parameter
  surrogate_loss1 = rho_s * advantages
  surrogate_loss2 = jnp.clip(rho_s, 1 - clipping_epsilon, 1 + clipping_epsilon) * advantages

  # TODO: PPO policy loss is the negative mean of the minimum of both surrogates
  policy_loss = -jnp.mean(jnp.minimum(surrogate_loss1, surrogate_loss2))

  # TODO: Implement value function loss
  # Hint: Mean squared error between vs (targets) and baseline (predictions)
  # Scale by 0.5 * 0.5 as in the original implementation
  v_error =(vs - baseline)**2
  v_loss = 0.25*jnp.mean(v_error) # 0.5 seems to be faster

  # TODO: Implement entropy loss for exploration
  # Hint: Use parametric_action_distribution.entropy() and multiply by entropy_cost
  # Make it negative since we want to maximize entropy (minimize negative entropy)
  entropy = parametric_action_distribution.entropy(policy_logits, rng)
  entropy_loss = -entropy_cost*jnp.mean(entropy)

  # TODO: Combine all loss components
  total_loss = policy_loss + v_loss + entropy_loss
  
  return total_loss, {
      'total_loss': total_loss,
      'policy_loss': policy_loss,
      'v_loss': v_loss,
      'entropy_loss': entropy_loss,
  }

In [ ]:
import os
import subprocess
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
import functools
from brax.training.agents.ppo import networks as ppo_networks
from mujoco_playground import wrapper
from mujoco_playground import registry
from mujoco_playground.config import locomotion_params
from brax.training.agents.ppo import losses as ppo_losses
from IPython.display import HTML, clear_output
import mujoco
import jax
import jax.numpy as jp
import cv2
import custom_ppo_train
from utils import render_video_during_training, evaluate_policy
import mediapy as media



env_cfg = default_config()

x_data, y_data, y_dataerr = [], [], []
times = [datetime.now()]

# Store the current policy for video rendering
current_policy = None

num_timesteps = 10

def progress(num_steps, metrics):
    clear_output(wait=True)

    times.append(datetime.now())
    x_data.append(num_steps)
    y_data.append(metrics["eval/episode_reward"])
    y_dataerr.append(metrics["eval/episode_reward_std"])

    plt.xlim([0, num_timesteps * 1.25])
    plt.xlabel("# environment steps")
    plt.ylabel("reward per episode")
    plt.title(f"y={y_data[-1]:.3f}")
    plt.errorbar(x_data, y_data, yerr=y_dataerr, color="blue")
    
    # Save the plot to a file instead of displaying it
    display(plt.gcf())

    # # Render video if we have a current policy
    # if current_policy is not None:
    #     render_video_during_training(current_policy, num_steps, jit_step, jit_reset, env_cfg, eval_env_for_video)

ppo_params = locomotion_params.brax_ppo_config('Go1JoystickRoughTerrain')
ppo_training_params = dict(ppo_params)

ppo_training_params["num_evals"] = 2 # Reduce for final training for less feedback.
ppo_training_params["num_timesteps"] = 10  # Total number of training steps (originally 200000000)
 
network_factory = ppo_networks.make_ppo_networks

if "network_factory" in ppo_params:
    del ppo_training_params["network_factory"]
    network_factory = functools.partial(
        ppo_networks.make_ppo_networks,
        **ppo_params.network_factory
    )
print(ppo_training_params)


# Create a policy parameters callback to capture the current policy
def policy_params_callback(_, make_policy_fn, params):
    # Update the current policy for video rendering
    global current_policy
    current_policy = make_policy_fn(params, deterministic=True)
    
train_fn = functools.partial(
        custom_ppo_train.train,
        **ppo_training_params,
        network_factory=network_factory,
        # randomization_fn=randomizer,
        progress_fn=progress,
        policy_params_fn=policy_params_callback,
)

In [ ]:
print("Starting training...")
make_policy, params, _ = train_fn(environment=env,
                                  eval_env=env,
                                  wrap_env_fn=wrapper.wrap_for_brax_training,
                                  compute_custom_ppo_loss_fn=compute_custom_ppo_loss
                                 )
print("Training completed.")